# Generate full emissions portfolio from projected Kyoto gas baskets
Updated to include announcements as of COP26, 5/11/21. Including an analysis of full impact of Net Zero commitments on emissions.

NOTE: Most of this file is unnecessary - only the file sr15_cleaned.csv is needed from this!

# Part 1: import and clean data

In [1]:
import sys
stdout = sys.stdout

In [2]:
import country_converter
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
import pyam
import re

<IPython.core.display.Javascript object>

In [3]:
from silicone.time_projectors import ExtendLatestTimeQuantile
import silicone.multiple_infillers as mi
import silicone.database_crunchers as dc
from silicone.utils import download_or_load_sr15, convert_units_to_MtCO2_equiv, _construct_consistent_values

In [4]:
version_no = "23"
version_no = version_no

# Import the data
Data is imported and put into pyam format. Names of regions are unified. Units of measurements are defined and cleaned. 

In [5]:
region_trans = {
    "OECD": "R5OECD90+EU",
    "Asia": "R5ASIA", 
    "Reforming Economies": 'R5REF', 
    "Latin America": 'R5LAM', 
    "Middle East and Africa": 'R5MAF', 
    "International bunkers": 'R5ROWO', 
    "World incl. Harmonisation": 'World', 
    "Harmonisation factor": "Harmonisation factor"
}

In [6]:
variable = "Emissions|Kyoto Gases (AR6-GWP100)"
metricname = "AR6GWP100"
kyexaf = variable + "|Excluding CO2 AFOLU"
unit = "Mt CO2-equiv/yr"
co2unit = "Mt CO2/yr"

In [7]:
# Read in and prepare the SR1.5 database. SR1.5 has more scenarios
sr15_file = "../input/complete_sr15.csv"
if not os.path.isfile(sr15_file):
    conn = pyam.iiasa.Connection("IXSE_SSP")
    sr15 = conn.query(timeslice=None)
    sr15.to_csv(sr15_file)
    sr15 = sr15.filter(region="World")
else:
    sr15 = pyam.IamDataFrame(sr15_file)

pyam - INFO: Running in a notebook, setting up a basic logging at level INFO
pyam.core - INFO: Reading file ..\input\complete_sr15.csv


In [8]:
co2 = "Emissions|CO2"
co2ind = "Emissions|CO2|Energy and Industrial Processes"
co2afolu = "Emissions|CO2|AFOLU"
kyotos = [co2ind, co2afolu, "Emissions|CH4", "Emissions|N2O", "Emissions|F-Gases"]

In [9]:
# Correct a difficult unit
sr15 = sr15.rename({"unit":{"kt HFC43-10/yr": "kt HFC4310mee/yr"}})

In [10]:
sr15 = sr15.filter(variable=["Emissions*", "Price|Carbon"]).filter(level="3+", keep=False).interpolate(2015)

In [11]:
messageg = "MESSAGE-GLOBIOM 1.0"
ssp2mes = sr15.filter(model=messageg, scenario="SSP2*", variable=variable)
ssp2mes = pyam.IamDataFrame(ssp2mes.data)

pyam.core - WARNING: Filtered IamDataFrame is empty!
pyam.utils - WARNING: Formatted data is empty!


In [12]:
regional_emtype = [co2ind, co2afolu, "Emissions|CH4", "Emissions|N2O", "Emissions|F-Gases"]
ssp2mes_reg = sr15.filter(scenario="SSP2*", model=messageg)
for region in ssp2mes_reg.region:
    if region != "World":
        ssp2mes_kyoto = _construct_consistent_values(
            variable, regional_emtype, convert_units_to_MtCO2_equiv(
                ssp2mes_reg.filter(region=region, variable=regional_emtype), metric_name=metricname
            )
        )
        ssp2mes_reg = ssp2mes_reg.append(ssp2mes_kyoto)
ssp2mes_reg.filter(variable=variable).data

,model,scenario,region,variable,unit,year,value
0,MESSAGE-GLOBIOM 1.0,SSP2-19,R5ASIA,Emissions|Kyoto Gases (AR6-GWP100),Mt CO2-equiv/yr,2005,15271.048346
1,MESSAGE-GLOBIOM 1.0,SSP2-19,R5ASIA,Emissions|Kyoto Gases (AR6-GWP100),Mt CO2-equiv/yr,2010,18375.065696
2,MESSAGE-GLOBIOM 1.0,SSP2-19,R5ASIA,Emissions|Kyoto Gases (AR6-GWP100),Mt CO2-equiv/yr,2015,19663.332920
3,MESSAGE-GLOBIOM 1.0,SSP2-19,R5ASIA,Emissions|Kyoto Gases (AR6-GWP100),Mt CO2-equiv/yr,2020,20951.600144
4,MESSAGE-GLOBIOM 1.0,SSP2-19,R5ASIA,Emissions|Kyoto Gases (AR6-GWP100),Mt CO2-equiv/yr,2030,12782.124974
...,...,...,...,...,...,...,...
355,MESSAGE-GLOBIOM 1.0,SSP2-Baseline,R5REF,Emissions|Kyoto Gases (AR6-GWP100),Mt CO2-equiv/yr,2060,4410.801147
356,MESSAGE-GLOBIOM 1.0,SSP2-Baseline,R5REF,Emissions|Kyoto Gases (AR6-GWP100),Mt CO2-equiv/yr,2070,4351.732653
357,MESSAGE-GLOBIOM 1.0,SSP2-Baseline,R5REF,Emissions|Kyoto Gases (AR6-GWP100),Mt CO2-equiv/yr,2080,4465.818453
358,MESSAGE-GLOBIOM 1.0,SSP2-Baseline,R5REF,Emissions|Kyoto Gases (AR6-GWP100),Mt CO2-equiv/yr,2090,4624.085038


In [13]:
ssp2mes_reg.check_aggregate_region(variable=variable)

pyam.core - INFO: Variable 'Emissions|Kyoto Gases (AR6-GWP100)' does not exist in region 'World'!


In [14]:
ssp2mes_reg.region

['R5ASIA', 'R5LAM', 'R5MAF', 'R5OECD90+EU', 'R5REF', 'World']

In [15]:
unit_lookup = sr15.data[["variable", "unit"]].drop_duplicates().set_index("variable")

In [16]:
# Import historical database
histall = pd.read_excel("../input/VanMarle2017Totals.xlsx", skiprows=1)
del histall["Unnamed: 13"]
del histall["Unnamed: 14"]
del histall["Unnamed: 15"]
histall = histall.rename(columns={"Scenario (History)": "Scenario"})
histall["Variable"] = histall["Variable"].str.replace("CMIP6 ", "")
histall["Scenario"] = "History"
histall["Region"] = histall["Region"].str.replace(".2", "").replace("R5OECD", "R5OECD90+EU")
histall = pyam.IamDataFrame(histall)

C:\Users\rlamboll\Anaconda3\envs\silicone-1\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == '__main__':


In [17]:
kyoto_gases = [
        'Emissions|C2F6', 
        'Emissions|CF4',
        'Emissions|CH4',
        'Emissions|CO2',
        'Emissions|HFC',
        'Emissions|SF6',
        'Emissions|N2O'
]
fgases = ["Emissions|HFC", 'Emissions|PFC|C2F6', 'EmissionsPFC|CF4', 'Emissions|SF6']
pfcs = ['Emissions|PFC|C2F6', 'Emissions|PFC|CF4']
histkyoto = _construct_consistent_values(
    variable, 
    kyoto_gases, 
    convert_units_to_MtCO2_equiv(histall.filter(region="World", variable=kyoto_gases), metric_name=metricname)
)
histall = histall.rename(variable={"Emissions|CF4":"Emissions|PFC|CF4", 'Emissions|C2F6':'Emissions|PFC|C2F6'})
histkyoto.timeseries()
histall = histall.append(histkyoto)
histnonafolu = histall.subtract(
    variable,
    co2afolu,
    name=kyexaf,
    ignore_units=True
)
histnonafolu = histnonafolu.data
histnonafolu["unit"] = unit
histnonafolu = pyam.IamDataFrame(histnonafolu)
histco2eni = histall.subtract(
    co2,
    co2afolu,
    name=co2ind,
    ignore_units=True
).data
histco2eni["unit"] = "Mt CO2/yr"
histco2eni = pyam.IamDataFrame(histco2eni)
histfgas = _construct_consistent_values(
    "Emissions|F-Gases", 
    fgases, 
    convert_units_to_MtCO2_equiv(histall.filter(region="World", variable=fgases), metric_name=metricname)
)
histpfc = _construct_consistent_values(
    "Emissions|PFC", 
    pfcs, 
    convert_units_to_MtCO2_equiv(histall.filter(region="World", variable=pfcs), metric_name=metricname)
)
histall = histall.append(histnonafolu).append(histco2eni).append(histfgas).append(histpfc)

In [18]:
histall.timeseries()

1990  \
model   scenario region variable           unit                      
History History  R5ASIA Emissions|BC       Mt BC/yr       2.943000   
                        Emissions|CH4      Mt CH4/yr     92.858640   
                        Emissions|CO       Mt CO/yr     299.179721   
                        Emissions|CO2      Mt CO2/yr   4089.048733   
                        Emissions|NH3      Mt NH3/yr     23.803027   
...                                                            ...   
                 World  Emissions|PFC|C2F6 kt C2F6/yr     2.110000   
                        Emissions|PFC|CF4  kt CF4/yr     14.810000   
                        Emissions|SF6      kt SF6/yr      4.880000   
                        Emissions|Sulfur   Mt SO2/yr    134.154791   
                        Emissions|VOC      Mt VOC/yr    208.103495   

                                                              1995  \
model   scenario region variable           unit                      
History History  R5ASIA Emissions|BC       Mt BC/yr       3.213000   
                        Emissions|CH4      Mt CH4/yr     99.653799   
                        Emissions|CO       Mt CO/yr     299.379993   
                        Emissions|CO2      Mt CO2/yr   5507.514650   
                        Emissions|NH3      Mt NH3/yr     25.299512   
...                                                            ...   
                 World  Emissions|PFC|C2F6 kt C2F6/yr     2.200000   
                        Emissions|PFC|CF4  kt CF4/yr     10.680000   
                        Emissions|SF6      kt SF6/yr      5.630000   
                        Emissions|Sulfur   Mt SO2/yr    121.397386   
                        Emissions|VOC      Mt VOC/yr    216.248905   

                                                              2000  \
model   scenario region variable           unit                      
History History  R5ASIA Emissions|BC       Mt BC/yr       3.177000   
                        Emissions|CH4      Mt CH4/yr    104.168651   
                        Emissions|CO       Mt CO/yr     308.268244   
                        Emissions|CO2      Mt CO2/yr   6200.737158   
                        Emissions|NH3      Mt NH3/yr     26.153645   
...                                                            ...   
                 World  Emissions|PFC|C2F6 kt C2F6/yr     2.850000   
                        Emissions|PFC|CF4  kt CF4/yr      9.820000   
                        Emissions|SF6      kt SF6/yr      4.860000   
                        Emissions|Sulfur   Mt SO2/yr    111.148729   
                        Emissions|VOC      Mt VOC/yr    195.329257   

                                                              2005  \
model   scenario region variable           unit                      
History History  R5ASIA Emissions|BC       Mt BC/yr       3.910000   
                        Emissions|CH4      Mt CH4/yr    124.903493   
                        Emissions|CO       Mt CO/yr     354.636749   
                        Emissions|CO2      Mt CO2/yr   9237.420318   
                        Emissions|NH3      Mt NH3/yr     28.538322   
...                                                            ...   
                 World  Emissions|PFC|C2F6 kt C2F6/yr     2.260000   
                        Emissions|PFC|CF4  kt CF4/yr     10.390000   
                        Emissions|SF6      kt SF6/yr      5.790000   
                        Emissions|Sulfur   Mt SO2/yr    124.980692   
                        Emissions|VOC      Mt VOC/yr    218.788737   

                                                               2010  \
model   scenario region variable           unit                       
History History  R5ASIA Emissions|BC       Mt BC/yr        4.529000   
                        Emissions|CH4      Mt CH4/yr     145.036835   
                        Emissions|CO       Mt CO/yr      383.928799   
                        Emissions|CO2      Mt CO2/yr   12471.21

# Save relevant values


In [19]:
out_folder = f"../output/processed_files/{version_no}/"
os.makedirs(out_folder, exist_ok=True)
sr15.to_csv(out_folder + "sr15_cleaned.csv")
histall.to_csv(out_folder + "histall_cleaned.csv")